In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os
import random
import sys
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from tqdm import tqdm
from sklearn.metrics import log_loss

from sklearn.decomposition import PCA

import numpy as np # linear algebra
import pandas as pd 

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


In [2]:
from sklearn.preprocessing import QuantileTransformer

In [3]:

data_dir = '../data/01_raw'
os.listdir(data_dir)

['train_targets_scored.csv',
 'sample_submission.csv',
 '.gitkeep',
 'train_drug.csv',
 'train_features.csv',
 'test_features.csv',
 'train_targets_nonscored.csv']

In [4]:
# Parameters
no_ctl = True
scale = "rankgauss"
decompo = "PCA"
ncompo_genes = 80
ncompo_cells = 10
encoding = "dummy"
variance_threshold = .7

In [5]:
train_features = pd.read_csv(data_dir+'/train_features.csv')
train_targets_scored = pd.read_csv(data_dir+'/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir+'/train_targets_nonscored.csv')
test_targets_nonscored = pd.read_csv('sub_nscored.csv')

test_features = pd.read_csv(data_dir+'/test_features.csv')
sample_submission = pd.read_csv(data_dir+'/sample_submission.csv')
drug = pd.read_csv(data_dir+'/train_drug.csv')

In [6]:
#add train and test nonscored as features
train_features = train_features.merge(train_targets_nonscored,on="sig_id")
test_features = test_features.merge(test_targets_nonscored,on="sig_id")

In [7]:
targets_scored = train_targets_scored.columns[1:]
targets_nscored = train_targets_nonscored.columns[1:]
scored = train_targets_scored.merge(drug, on='sig_id', how='left') 

In [8]:
if no_ctl:
    # cp_type == ctl_vehicle
    print("not_ctl")
    train_features = train_features[train_features["cp_type"] != "ctl_vehicle"]
    test_features = test_features[test_features["cp_type"] != "ctl_vehicle"]
    train_targets_scored = train_targets_scored.iloc[train_features.index]
    train_features.reset_index(drop = True, inplace = True)
    test_features.reset_index(drop = True, inplace = True)
    train_targets_scored.reset_index(drop = True, inplace = True)

not_ctl


In [9]:
seed = 42


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    
seed_everything(seed)

In [10]:
# #Indiquer si valeur dans le range max, min

# import seaborn as sns
# data = pd.concat([train_features,test_features],axis=0)

# sns.distplot(data[data["cp_type"] == "ctl_vehicle"]["c-4"],label="normal")

# sns.distplot(data[data["cp_type"] == "trt_cp"]["c-4"],label="treated")
# plt.legend()
# plt.show()

In [11]:
# sum_targets = train_targets_scored[[c for c in train_targets_scored.columns if (c != "sig_id")]].sum().reset_index()

In [12]:
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_inhibitor",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_activator",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_agonist",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_antagonist",""))

In [13]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [14]:
data_all = pd.concat([train_features, test_features], ignore_index = True)
# cols_numeric = [feat for feat in list(data_all.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]
# mask = (data_all[cols_numeric].var() >= variance_threshould).values
# tmp = data_all[cols_numeric].loc[:, mask]
# data_all = pd.concat([data_all[["sig_id", "cp_type", "cp_time", "cp_dose"]], tmp], axis = 1)
cols_numeric = [feat for feat in list(data_all.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"] +\
                                                                       list(train_targets_nonscored.columns)]

In [15]:
# from scipy import stats

# train_features[GENES].apply(lambda x : stats.moment(x,moment=5),axis=1)

In [16]:
#True gauss rank
import cupy as cp
from cupyx.scipy.special import erfinv
epsilon = 1e-6

for k in (cols_numeric):
    r_gpu = cp.array(data_all.loc[:,k])
    r_gpu = r_gpu.argsort().argsort()
    r_gpu = (r_gpu/r_gpu.max()-0.5)*2 
    r_gpu = cp.clip(r_gpu,-1+epsilon,1-epsilon)
    r_gpu = erfinv(r_gpu) 
    data_all.loc[:,k] = cp.asnumpy( r_gpu * np.sqrt(2) )

In [17]:
# #Quantiletransformer
# for col in (cols_numeric):

#     transformer = QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
#     vec_len = len(data_all[col].values)
#     raw_vec = data_all[col].values.reshape(vec_len, 1)
#     transformer.fit(raw_vec)

#     data_all[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]

In [18]:
#PCA
if decompo == "PCA":
    print("PCA")
    GENES = [col for col in data_all.columns if col.startswith("g-")]
    CELLS = [col for col in data_all.columns if col.startswith("c-")]
    
    pca_genes = PCA(n_components = ncompo_genes,
                    random_state = seed).fit_transform(data_all[GENES])
    pca_cells = PCA(n_components = ncompo_cells,
                    random_state = seed).fit_transform(data_all[CELLS])
    
    pca_genes = pd.DataFrame(pca_genes, columns = [f"pca_g-{i}" for i in range(ncompo_genes)])
    pca_cells = pd.DataFrame(pca_cells, columns = [f"pca_c-{i}" for i in range(ncompo_cells)])
    data_all = pd.concat([data_all, pca_genes, pca_cells], axis = 1)
else:
    pass

PCA


In [19]:
# Encoding
if encoding == "lb":
    print("Label Encoding")
    for feat in ["cp_time", "cp_dose"]:
        data_all[feat] = LabelEncoder().fit_transform(data_all[feat])
elif encoding == "dummy":
    print("One-Hot")
    data_all = pd.get_dummies(data_all, columns = ["cp_time", "cp_dose"])

One-Hot


In [20]:
GENES = [col for col in data_all.columns if col.startswith("g-")]
CELLS = [col for col in data_all.columns if col.startswith("c-")]

for stats in ["sum", "mean", "std", "kurt", "skew"]:
# for stats in ["sum",  "std"]:
    data_all["g_" + stats] = getattr(data_all[GENES], stats)(axis = 1)
    data_all["c_" + stats] = getattr(data_all[CELLS], stats)(axis = 1)

for stats in ["sum", "mean", "std", "kurt", "skew"]:
# for stats in ["sum"]:
    data_all["gc_" + stats] = getattr(data_all[GENES + CELLS], stats)(axis = 1)

In [21]:
# #Kmeans
# from sklearn.cluster import KMeans
# def fe_cluster2(data, n_clusters_c = 3, n_clusters_g = 20, seed = 42):
#     kmeans_c = KMeans(n_clusters = n_clusters_c, random_state = seed).fit(data.loc[:,CELLS])
#     data['cluster_c'] = kmeans_c.labels_
#     data = pd.get_dummies(data, columns = ['cluster_c'])

#     kmeans_g = KMeans(n_clusters = n_clusters_g, random_state = seed).fit(data.loc[:,GENES])
#     data['cluster_g'] = kmeans_g.labels_
#     data = pd.get_dummies(data, columns = ['cluster_g'])
#     return data
    
# data_all=fe_cluster2(data_all,n_clusters_c=3,n_clusters_g=20,seed=42)

In [22]:
from sklearn.feature_selection import VarianceThreshold

feat_cols = [c for c in data_all.columns if c not in ["sig_id", "cp_type"]]
var_thresh = VarianceThreshold(variance_threshold)  #<-- Update
data_feats = pd.DataFrame(var_thresh.fit_transform(data_all[feat_cols]))

In [23]:
# data_all = pd.concat([data_all["sig_id"],data_all[targets_nscored], data_feats],axis=1)
data_all = pd.concat([data_all["sig_id"], data_feats],axis=1)

In [24]:
# from sklearn.cluster import KMeans
# distortion = []
# for k in range(1,10):
#     kmeans = KMeans(n_clusters = k, random_state = seed).fit(data_all)
#     distortion += [kmeans.inertia_]

In [25]:
# import matplotlib.pyplot as plt
# plt.plot(range(1,10),distortion)

In [26]:
data_all.shape

(25572, 965)

In [27]:
# train_df and test_df
try:
    train_targets_scored.drop("sig_id", axis = 1, inplace = True)
    train_targets_nonscored.drop("sig_id", axis = 1, inplace = True)
except:
    pass
train_df = data_all[: train_features.shape[0]]
train_df.reset_index(drop = True, inplace = True)
# The following line it's a bad practice in my opinion, targets on train set
#train_df = pd.concat([train_df, targets], axis = 1)
test_df = data_all[train_df.shape[0]: ]
test_df.reset_index(drop = True, inplace = True)

In [28]:
print(f"train_df.shape: {train_df.shape}")
print(f"test_df.shape: {test_df.shape}")

train_df.shape: (21948, 965)
test_df.shape: (3624, 965)


In [29]:
X_test = test_df.values
print(f"X_test.shape: {X_test.shape}")

X_test.shape: (3624, 965)


In [30]:
import torch
import numpy as np
from scipy.sparse import csc_matrix
import time
from abc import abstractmethod
#from pytorch_tabnet import tab_network
from pytorch_tabnet.multiclass_utils import unique_labels
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
from torch.nn.utils import clip_grad_norm_
from pytorch_tabnet.utils import (PredictDataset,
                                  create_explain_matrix)
from sklearn.base import BaseEstimator
from torch.utils.data import DataLoader
from copy import deepcopy
import io
import json
from pathlib import Path
import shutil
import zipfile

In [31]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

def log_loss_multi(y_true, y_pred):
    M = y_true.shape[1]
    results = np.zeros(M)
    for i in range(M):
        results[i] = log_loss_score(y_true[:,i], y_pred[:,i])
    return results.mean()

def log_loss_score(actual, predicted,  eps=1e-15):

        """
        :param predicted:   The predicted probabilities as floats between 0-1
        :param actual:      The binary labels. Either 0 or 1.
        :param eps:         Log(0) is equal to infinity, so we need to offset our predicted values slightly by eps from 0 or 1
        :return:            The logarithmic loss between between the predicted probability assigned to the possible outcomes for item i, and the actual outcome.
        """

        
        p1 = actual * np.log(predicted+eps)
        p0 = (1-actual) * np.log(1-predicted+eps)
        loss = p0 + p1

        return -loss.mean()

In [32]:
class MoADataset(Dataset):
    """
    Format for numpy array
    Parameters
    ----------
    X : 2D array
        The input matrix
    y : 2D array
        The one-hot encoded target
    """

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [33]:
def create_dataloaders(
    X_train, y_scored_train, X_valid,y_valid, weights, batch_size, num_workers, drop_last, pin_memory=True
):
    """
    Create dataloaders with or wihtout subsampling depending on weights and balanced.
    Parameters
    ----------
    X_train : np.ndarray
        Training data
    y_train : np.array
        Mapped Training targets
    eval_set : list of tuple
        List of eval tuple set (X, y)
    weights : either 0, 1, dict or iterable
        if 0 (default) : no weights will be applied
        if 1 : classification only, will balanced class with inverse frequency
        if dict : keys are corresponding class values are sample weights
        if iterable : list or np array must be of length equal to nb elements
                      in the training set
    batch_size : int
        how many samples per batch to load
    num_workers : int
        how many subprocesses to use for data loading. 0 means that the data
        will be loaded in the main process
    drop_last : bool
        set to True to drop the last incomplete batch, if the dataset size is not
        divisible by the batch size. If False and the size of dataset is not
        divisible by the batch size, then the last batch will be smaller
    pin_memory : bool
        Whether to pin GPU memory during training
    Returns
    -------
    train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
        Training and validation dataloaders
    """

    if isinstance(weights, int):
        if weights == 0:
            need_shuffle = True
            sampler = None
        elif weights == 1:
            need_shuffle = False
            class_sample_count = np.array(
                [len(np.where(y_train == t)[0]) for t in np.unique(y_scored_train)]
            )

            weights = 1.0 / class_sample_count

            samples_weight = np.array([weights[t] for t in y_scored_train])

            samples_weight = torch.from_numpy(samples_weight)
            samples_weight = samples_weight.double()
            sampler = WeightedRandomSampler(samples_weight, len(samples_weight))
        else:
            raise ValueError("Weights should be either 0, 1, dictionnary or list.")
    elif isinstance(weights, dict):
        # custom weights per class
        need_shuffle = False
        samples_weight = np.array([weights[t] for t in y_scored_train])
        sampler = WeightedRandomSampler(samples_weight, len(samples_weight))
    else:
        # custom weights
        if len(weights) != len(y_scored_train):
            raise ValueError("Custom weights should match number of train samples.")
        need_shuffle = False
        samples_weight = np.array(weights)
        sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

    train_dataloader = DataLoader(
        MoADataset(X_train.astype(np.float32), y_scored_train),
        batch_size=batch_size,
        sampler=sampler,
        shuffle=need_shuffle,
        num_workers=num_workers,
        drop_last=drop_last,
        pin_memory=pin_memory
    )

    # valid_dataloaders = []
    # for X, y in [(X_valid,y_valid)]:
    #     valid_dataloaders.append(
    #         DataLoader(
    #             ValidDataset(X.astype(np.float32), y),
    #             batch_size=batch_size,
    #             shuffle=False,
    #             num_workers=num_workers,
    #             pin_memory=pin_memory
    #         )
    #     )

    valid_dataloaders = DataLoader(
        MoADataset(X_valid.astype(np.float32), y_valid),
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory
    )


    return train_dataloader, valid_dataloaders

In [34]:
import torch
from torch.nn import Linear, BatchNorm1d, ReLU, PReLU,LeakyReLU
import numpy as np
from pytorch_tabnet import sparsemax


def initialize_non_glu(module, input_dim, output_dim):
    gain_value = np.sqrt((input_dim+output_dim)/np.sqrt(4*input_dim))
    torch.nn.init.xavier_normal_(module.weight, gain=gain_value)
    # torch.nn.init.zeros_(module.bias)
    return


def initialize_glu(module, input_dim, output_dim):
    gain_value = np.sqrt((input_dim+output_dim)/np.sqrt(input_dim))
    torch.nn.init.xavier_normal_(module.weight, gain=gain_value)
    # torch.nn.init.zeros_(module.bias)
    return


class GBN(torch.nn.Module):
    """
        Ghost Batch Normalization
        https://arxiv.org/abs/1705.08741
    """

    def __init__(self, input_dim, virtual_batch_size=128, momentum=0.01):
        super(GBN, self).__init__()

        self.input_dim = input_dim
        self.virtual_batch_size = virtual_batch_size
        self.bn = BatchNorm1d(self.input_dim, momentum=momentum)
        # self.dropout = nn.Dropout(.2)

    def forward(self, x):
        chunks = x.chunk(int(np.ceil(x.shape[0] / self.virtual_batch_size)), 0)
        res = [self.bn(x_) for x_ in chunks]

        #return self.dropout(torch.cat(res, dim=0))
        return torch.cat(res, dim=0)


class TabNetNoEmbeddings(torch.nn.Module):
    def __init__(self, input_dim, output_dim,
                 n_d=8, n_a=8,
                 n_steps=3, gamma=1.3,
                 n_independent=2, n_shared=2, epsilon=1e-15,
                 virtual_batch_size=128, momentum=0.02,
                 mask_type="sparsemax"):
        """
        Defines main part of the TabNet network without the embedding layers.
        Parameters
        ----------
        input_dim : int
            Number of features
        output_dim : int or list of int for multi task classification
            Dimension of network output
            examples : one for regression, 2 for binary classification etc...
        n_d : int
            Dimension of the prediction  layer (usually between 4 and 64)
        n_a : int
            Dimension of the attention  layer (usually between 4 and 64)
        n_steps : int
            Number of sucessive steps in the newtork (usually betwenn 3 and 10)
        gamma : float
            Float above 1, scaling factor for attention updates (usually betwenn 1.0 to 2.0)
        n_independent : int
            Number of independent GLU layer in each GLU block (default 2)
        n_shared : int
            Number of independent GLU layer in each GLU block (default 2)
        epsilon : float
            Avoid log(0), this should be kept very low
        virtual_batch_size : int
            Batch size for Ghost Batch Normalization
        momentum : float
            Float value between 0 and 1 which will be used for momentum in all batch norm
        mask_type : str
            Either "sparsemax" or "entmax" : this is the masking function to use
        """
        super(TabNetNoEmbeddings, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.is_multi_task = isinstance(output_dim, list)
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.epsilon = epsilon
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.virtual_batch_size = virtual_batch_size
        self.mask_type = mask_type
        self.initial_bn = BatchNorm1d(self.input_dim, momentum=0.01)

        if self.n_shared > 0:
            shared_feat_transform = torch.nn.ModuleList()

            for i in range(self.n_shared):
                if i == 0:
                    shared_feat_transform.append(Linear(self.input_dim,
                                                        2*(n_d + n_a),
                                                        bias=False))
                else:
                    shared_feat_transform.append(Linear(n_d + n_a, 2*(n_d + n_a), bias=False))

        else:
            shared_feat_transform = None

        self.initial_splitter = FeatTransformer(self.input_dim, n_d+n_a, shared_feat_transform,
                                                n_glu_independent=self.n_independent,
                                                virtual_batch_size=self.virtual_batch_size,
                                                momentum=momentum)

        self.feat_transformers = torch.nn.ModuleList()
        self.att_transformers = torch.nn.ModuleList()


        for step in range(n_steps):
            transformer = FeatTransformer(self.input_dim, n_d+n_a, shared_feat_transform,
                                          n_glu_independent=self.n_independent,
                                          virtual_batch_size=self.virtual_batch_size,
                                          momentum=momentum)

            attention = AttentiveTransformer(n_a, self.input_dim,
                                             virtual_batch_size=self.virtual_batch_size,
                                             momentum=momentum,
                                             mask_type=self.mask_type)

            self.feat_transformers.append(transformer)
            self.att_transformers.append(attention)

        if self.is_multi_task:
            self.multi_task_mappings = torch.nn.ModuleList()
            for task_dim in output_dim:
                task_mapping = Linear(n_d, task_dim, bias=False)
                initialize_non_glu(task_mapping, n_d, task_dim)
                self.multi_task_mappings.append(task_mapping)

            # self.multi_task_mappings2 = torch.nn.ModuleList()
            # for task_dim in output_dim2:
            #     task_mapping = Linear(n_d2, task_dim, bias=False)
            #     initialize_non_glu(task_mapping, n_d2, task_dim)
            #     self.multi_task_mappings2.append(task_mapping)
        else:
            self.final_mapping = Linear(n_d, output_dim, bias=False)
            initialize_non_glu(self.final_mapping, n_d, output_dim)
            # self.final_mapping2 = Linear(n_d2, output_dim2, bias=False)
            # initialize_non_glu(self.final_mapping2, n_d2, output_dim2)

    def forward(self, x):
        res = 0
        x = self.initial_bn(x)

        prior = torch.ones(x.shape).to(x.device)

        M_loss = 0
        att = self.initial_splitter(x)[:, self.n_d:]
        
        for step in range(self.n_steps):
            M = self.att_transformers[step](prior, att)

            M_loss += torch.mean(torch.sum(torch.mul(M, torch.log(M+self.epsilon)),
                                           dim=1))

            # update prior
            prior = torch.mul(self.gamma - M, prior)
            # output
            masked_x = torch.mul(M, x)
            out = self.feat_transformers[step](masked_x)
            # d = ReLU()(out[:, :self.n_d])
            d = PReLU().to(x.device)(out[:, :self.n_d])
            res = torch.add(res, d)
            # update attention
            att = out[:, self.n_d:]

        M_loss /= self.n_steps

        if self.is_multi_task:
            # Result will be in list format
            out = []
            for task_mapping in self.multi_task_mappings:
                out.append(task_mapping(res))

        else:
            out = self.final_mapping(res)
        return out, M_loss
        # return out1, M_loss

    def forward_masks(self, x):
        x = self.initial_bn(x)

        prior = torch.ones(x.shape).to(x.device)
        M_explain = torch.zeros(x.shape).to(x.device)
        att = self.initial_splitter(x)[:, self.n_d:]
        masks = {}

        for step in range(self.n_steps):
            M = self.att_transformers[step](prior, att)
            masks[step] = M
            # update prior
            prior = torch.mul(self.gamma - M, prior)
            # output
            masked_x = torch.mul(M, x)
            out = self.feat_transformers[step](masked_x)
            # d = ReLU()(out[:, :self.n_d])
            d = PReLU().to(x.device)(out[:, :self.n_d])
            # explain
            step_importance = torch.sum(d, dim=1)
            M_explain += torch.mul(M, step_importance.unsqueeze(dim=1))
            # update attention
            att = out[:, self.n_d:]

        return M_explain, masks


class TabNet(torch.nn.Module):
    def __init__(self, input_dim, output_dim, n_d=8, n_a=8,
                 n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1,
                 n_independent=2, n_shared=2, epsilon=1e-15,
                 virtual_batch_size=128, momentum=0.02, device_name='auto',
                 mask_type="sparsemax"):
        """
        Defines TabNet network
        Parameters
        ----------
        input_dim : int
            Initial number of features
        output_dim : int
            Dimension of network output
            examples : one for regression, 2 for binary classification etc...
        n_d : int
            Dimension of the prediction  layer (usually between 4 and 64)
        n_a : int
            Dimension of the attention  layer (usually between 4 and 64)
        n_steps : int
            Number of sucessive steps in the newtork (usually betwenn 3 and 10)
        gamma : float
            Float above 1, scaling factor for attention updates (usually betwenn 1.0 to 2.0)
        cat_idxs : list of int
            Index of each categorical column in the dataset
        cat_dims : list of int
            Number of categories in each categorical column
        cat_emb_dim : int or list of int
            Size of the embedding of categorical features
            if int, all categorical features will have same embedding size
            if list of int, every corresponding feature will have specific size
        n_independent : int
            Number of independent GLU layer in each GLU block (default 2)
        n_shared : int
            Number of independent GLU layer in each GLU block (default 2)
        epsilon : float
            Avoid log(0), this should be kept very low
        virtual_batch_size : int
            Batch size for Ghost Batch Normalization
        momentum : float
            Float value between 0 and 1 which will be used for momentum in all batch norm
        device_name : {'auto', 'cuda', 'cpu'}
        mask_type : str
            Either "sparsemax" or "entmax" : this is the masking function to use
        """
        super(TabNet, self).__init__()
        self.cat_idxs = cat_idxs or []
        self.cat_dims = cat_dims or []
        self.cat_emb_dim = cat_emb_dim

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.epsilon = epsilon
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.mask_type = mask_type

        if self.n_steps <= 0:
            raise ValueError("n_steps should be a positive integer.")
        if self.n_independent == 0 and self.n_shared == 0:
            raise ValueError("n_shared and n_independant can't be both zero.")

        self.virtual_batch_size = virtual_batch_size
        self.embedder = EmbeddingGenerator(input_dim, cat_dims, cat_idxs, cat_emb_dim)
        self.post_embed_dim = self.embedder.post_embed_dim
        self.tabnet = TabNetNoEmbeddings(self.post_embed_dim, output_dim,n_d, n_a, n_steps,
                                         gamma, n_independent, n_shared, epsilon,
                                         virtual_batch_size, momentum, mask_type)

        # Defining device
        if device_name == 'auto':
            if torch.cuda.is_available():
                device_name = 'cuda'
            else:
                device_name = 'cpu'
        self.device = torch.device(device_name)
        self.to(self.device)

    def forward(self, x):
        x = self.embedder(x)
        return self.tabnet(x)

    def forward_masks(self, x):
        x = self.embedder(x)
        return self.tabnet.forward_masks(x)


class AttentiveTransformer(torch.nn.Module):
    def __init__(self, input_dim, output_dim,
                 virtual_batch_size=128,
                 momentum=0.02,
                 mask_type="sparsemax"):
        """
        Initialize an attention transformer.
        Parameters
        ----------
        input_dim : int
            Input size
        output_dim : int
            Outpu_size
        virtual_batch_size : int
            Batch size for Ghost Batch Normalization
        momentum : float
            Float value between 0 and 1 which will be used for momentum in batch norm
        mask_type : str
            Either "sparsemax" or "entmax" : this is the masking function to use
        """
        super(AttentiveTransformer, self).__init__()
        self.fc = Linear(input_dim, output_dim, bias=False)
        initialize_non_glu(self.fc, input_dim, output_dim)
        self.bn = GBN(output_dim, virtual_batch_size=virtual_batch_size,
                      momentum=momentum)

        if mask_type == "sparsemax":
            # Sparsemax
            self.selector = sparsemax.Sparsemax(dim=-1)
        elif mask_type == "entmax":
            # Entmax
            self.selector = sparsemax.Entmax15(dim=-1)
        else:
            raise NotImplementedError("Please choose either sparsemax" +
                                      "or entmax as masktype")

    def forward(self, priors, processed_feat):
        x = self.fc(processed_feat)
        x = self.bn(x)
        x = torch.mul(x, priors)
        x = self.selector(x)
        return x


class FeatTransformer(torch.nn.Module):
    def __init__(self, input_dim, output_dim, shared_layers, n_glu_independent,
                 virtual_batch_size=128, momentum=0.02):
        super(FeatTransformer, self).__init__()
        """
        Initialize a feature transformer.
        Parameters
        ----------
        input_dim : int
            Input size
        output_dim : int
            Outpu_size
        shared_layers : torch.nn.ModuleList
            The shared block that should be common to every step
        n_glu_independant : int
            Number of independent GLU layers
        virtual_batch_size : int
            Batch size for Ghost Batch Normalization within GLU block(s)
        momentum : float
            Float value between 0 and 1 which will be used for momentum in batch norm
        """

        params = {
            'n_glu': n_glu_independent,
            'virtual_batch_size': virtual_batch_size,
            'momentum': momentum
        }

        if shared_layers is None:
            # no shared layers
            self.shared = torch.nn.Identity()
            is_first = True
        else:
            self.shared = GLU_Block(input_dim, output_dim,
                                    first=True,
                                    shared_layers=shared_layers,
                                    n_glu=len(shared_layers),
                                    virtual_batch_size=virtual_batch_size,
                                    momentum=momentum)
            is_first = False

        if n_glu_independent == 0:
            # no independent layers
            self.specifics = torch.nn.Identity()
        else:
            spec_input_dim = input_dim if is_first else output_dim
            self.specifics = GLU_Block(spec_input_dim, output_dim,
                                       first=is_first,
                                       **params)

    def forward(self, x):
        x = self.shared(x)
        x = self.specifics(x)
        return x


class GLU_Block(torch.nn.Module):
    """
        Independant GLU block, specific to each step
    """

    def __init__(self, input_dim, output_dim, n_glu=2, first=False, shared_layers=None,
                 virtual_batch_size=128, momentum=0.02):
        super(GLU_Block, self).__init__()
        self.first = first
        self.shared_layers = shared_layers
        self.n_glu = n_glu
        self.glu_layers = torch.nn.ModuleList()

        params = {
            'virtual_batch_size': virtual_batch_size,
            'momentum': momentum
        }

        fc = shared_layers[0] if shared_layers else None
        self.glu_layers.append(GLU_Layer(input_dim, output_dim,
                                         fc=fc,
                                         **params))
        for glu_id in range(1, self.n_glu):
            fc = shared_layers[glu_id] if shared_layers else None
            self.glu_layers.append(GLU_Layer(output_dim, output_dim,
                                             fc=fc,
                                             **params))

    def forward(self, x):
        scale = torch.sqrt(torch.FloatTensor([0.5]).to(x.device))
        if self.first:  # the first layer of the block has no scale multiplication
            x = self.glu_layers[0](x)
            layers_left = range(1, self.n_glu)
        else:
            layers_left = range(self.n_glu)

        for glu_id in layers_left:
            x = torch.add(x, self.glu_layers[glu_id](x))
            x = x*scale
        return x


class GLU_Layer(torch.nn.Module):
    def __init__(self, input_dim, output_dim, fc=None,
                 virtual_batch_size=128, momentum=0.02):
        super(GLU_Layer, self).__init__()

        self.output_dim = output_dim
        if fc:
            self.fc = fc
        else:
            self.fc = Linear(input_dim, 2*output_dim, bias=False)
        initialize_glu(self.fc, input_dim, 2*output_dim)

        self.bn = GBN(2*output_dim, virtual_batch_size=virtual_batch_size,
                      momentum=momentum)


        #self.dropout = nn.Dropout(.1)

    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        #x = self.dropout(x)
        out = torch.mul(x[:, :self.output_dim], torch.sigmoid(x[:, self.output_dim:]))
        return out


class EmbeddingGenerator(torch.nn.Module):
    """
        Classical embeddings generator
    """

    def __init__(self, input_dim, cat_dims, cat_idxs, cat_emb_dim):
        """ This is an embedding module for an entier set of features
        Parameters
        ----------
        input_dim : int
            Number of features coming as input (number of columns)
        cat_dims : list of int
            Number of modalities for each categorial features
            If the list is empty, no embeddings will be done
        cat_idxs : list of int
            Positional index for each categorical features in inputs
        cat_emb_dim : int or list of int
            Embedding dimension for each categorical features
            If int, the same embdeding dimension will be used for all categorical features
        """
        super(EmbeddingGenerator, self).__init__()
        if cat_dims == [] or cat_idxs == []:
            self.skip_embedding = True
            self.post_embed_dim = input_dim
            return

        self.skip_embedding = False
        if isinstance(cat_emb_dim, int):
            self.cat_emb_dims = [cat_emb_dim]*len(cat_idxs)
        else:
            self.cat_emb_dims = cat_emb_dim

        # check that all embeddings are provided
        if len(self.cat_emb_dims) != len(cat_dims):
            msg = """ cat_emb_dim and cat_dims must be lists of same length, got {len(self.cat_emb_dims)}
                      and {len(cat_dims)}"""
            raise ValueError(msg)
        self.post_embed_dim = int(input_dim + np.sum(self.cat_emb_dims) - len(self.cat_emb_dims))

        self.embeddings = torch.nn.ModuleList()

        # Sort dims by cat_idx
        sorted_idxs = np.argsort(cat_idxs)
        cat_dims = [cat_dims[i] for i in sorted_idxs]
        self.cat_emb_dims = [self.cat_emb_dims[i] for i in sorted_idxs]

        for cat_dim, emb_dim in zip(cat_dims, self.cat_emb_dims):
            self.embeddings.append(torch.nn.Embedding(cat_dim, emb_dim))

        # record continuous indices
        self.continuous_idx = torch.ones(input_dim, dtype=torch.bool)
        self.continuous_idx[cat_idxs] = 0

    def forward(self, x):
        """
        Apply embdeddings to inputs
        Inputs should be (batch_size, input_dim)
        Outputs will be of size (batch_size, self.post_embed_dim)
        """
        if self.skip_embedding:
            # no embeddings required
            return x

        cols = []
        cat_feat_counter = 0
        for feat_init_idx, is_continuous in enumerate(self.continuous_idx):
            # Enumerate through continuous idx boolean mask to apply embeddings
            if is_continuous:
                cols.append(x[:, feat_init_idx].float().view(-1, 1))
            else:
                cols.append(self.embeddings[cat_feat_counter](x[:, feat_init_idx].long()))
                cat_feat_counter += 1
        # concat
        post_embeddings = torch.cat(cols, dim=1)
        return post_embeddings

In [35]:
class TabModel(BaseEstimator):
    def __init__(self, n_d=8, n_a=8, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1,
                 n_independent=2, n_shared=2, epsilon=1e-15,  momentum=0.02,
                 lambda_sparse=1e-3, seed=0,
                 clip_value=1, verbose=1,
                 optimizer_fn=torch.optim.Adam,
                 optimizer_params=dict(lr=2e-2),
                 scheduler_params=None, scheduler_fn=None,
                 mask_type="sparsemax",
                 input_dim=None, output_dim=None,
                 device_name='auto'):
        """ Class for TabNet model
        Parameters
        ----------
            device_name: str
                'cuda' if running on GPU, 'cpu' if not, 'auto' to autodetect
        """

        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.cat_idxs = cat_idxs
        self.cat_dims = cat_dims
        self.cat_emb_dim = cat_emb_dim
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.epsilon = epsilon
        self.momentum = momentum
        self.lambda_sparse = lambda_sparse
        self.clip_value = clip_value
        self.verbose = verbose
        self.optimizer_fn = optimizer_fn
        self.optimizer_params = optimizer_params
        self.device_name = device_name
        self.scheduler_params = scheduler_params
        self.scheduler_fn = scheduler_fn
        self.mask_type = mask_type
        self.input_dim = input_dim
        self.output_dim = output_dim

        #self.batch_size = 1024
        self.batch_size = 1024

        self.seed = seed
        torch.manual_seed(self.seed)
        # Defining device
        if device_name == 'auto':
            if torch.cuda.is_available():
                device_name = 'cuda'
            else:
                device_name = 'cpu'
        self.device = torch.device(device_name)
        print(f"Device used : {self.device}")

    @abstractmethod
    def construct_loaders(self, X_train, y_scored_train, X_valid, y_valid,
                          weights, batch_size, num_workers, drop_last):
        """
        Returns
        -------
        train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
            Training and validation dataloaders
        -------
        """
        raise NotImplementedError('users must define construct_loaders to use this base class')

    def init_network(
                     self,
                     input_dim,
                     output_dim,
                     n_d,
                     n_a,
                     n_steps,
                     gamma,
                     cat_idxs,
                     cat_dims,
                     cat_emb_dim,
                     n_independent,
                     n_shared,
                     epsilon,
                     virtual_batch_size,
                     momentum,
                     device_name,
                     mask_type,
                     ):
        self.network = TabNet(
            input_dim,
            output_dim,
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            cat_idxs=cat_idxs,
            cat_dims=cat_dims,
            cat_emb_dim=cat_emb_dim,
            n_independent=n_independent,
            n_shared=n_shared,
            epsilon=epsilon,
            virtual_batch_size=virtual_batch_size,
            momentum=momentum,
            device_name=device_name,
            mask_type=mask_type).to(self.device)

        self.reducing_matrix = create_explain_matrix(
            self.network.input_dim,
            self.network.cat_emb_dim,
            self.network.cat_idxs,
            self.network.post_embed_dim)

    def fit(self, X_train, y_scored_train, X_valid=None, y_valid=None, loss_fn=None,loss_tr=None,
            weights=0, max_epochs=100, patience=10, batch_size=1024,
            virtual_batch_size=128, num_workers=0, drop_last=False):
        """Train a neural network stored in self.network
        Using train_dataloader for training data and
        valid_dataloader for validation.
        Parameters
        ----------
            X_train: np.ndarray
                Train set
            y_train : np.array
                Train targets
            X_train: np.ndarray
                Train set
            y_train : np.array
                Train targets
            weights : bool or dictionnary
                0 for no balancing
                1 for automated balancing
                dict for custom weights per class
            max_epochs : int
                Maximum number of epochs during training
            patience : int
                Number of consecutive non improving epoch before early stopping
            batch_size : int
                Training batch size
            virtual_batch_size : int
                Batch size for Ghost Batch Normalization (virtual_batch_size < batch_size)
            num_workers : int
                Number of workers used in torch.utils.data.DataLoader
            drop_last : bool
                Whether to drop last batch during training
        """
        # update model name

        self.update_fit_params(X_train, y_scored_train, X_valid, y_valid, loss_fn,loss_tr,
                               weights, max_epochs, patience, batch_size,virtual_batch_size, num_workers, drop_last)


        train_dataloader, valid_dataloader = self.construct_loaders(X_train,
                                                                    y_scored_train,
                                                                    X_valid,
                                                                    y_valid,
                                                                    self.updated_weights,
                                                                    self.batch_size,
                                                                    self.num_workers,
                                                                    self.drop_last)

        self.init_network(
            input_dim=self.input_dim,
            output_dim=self.output_dim,
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            cat_idxs=self.cat_idxs,
            cat_dims=self.cat_dims,
            cat_emb_dim=self.cat_emb_dim,
            n_independent=self.n_independent,
            n_shared=self.n_shared,
            epsilon=self.epsilon,
            virtual_batch_size=self.virtual_batch_size,
            momentum=self.momentum,
            device_name=self.device_name,
            mask_type=self.mask_type
        )

        self.optimizer = self.optimizer_fn(self.network.parameters(),
                                           **self.optimizer_params)

        if self.scheduler_fn:
            self.scheduler = self.scheduler_fn(self.optimizer, **self.scheduler_params)
        else:
            self.scheduler = None

        self.losses_train = []
        self.losses_valid = []
        self.learning_rates = []
        self.metrics_train = []
        self.metrics_valid = []

        if self.verbose > 0:
            print("Will train until validation stopping metric",
                  f"hasn't improved in {self.patience} rounds.")
            msg_epoch = f'| EPOCH |  train  |   valid  | total time (s)'
            print('---------------------------------------')
            print(msg_epoch)

        total_time = 0
        while (self.epoch < self.max_epochs and self.patience_counter < self.patience):
            starting_time = time.time()
            # updates learning rate history
            self.learning_rates.append(self.optimizer.param_groups[-1]["lr"])

            fit_metrics = self.fit_epoch(train_dataloader, valid_dataloader)

            # leaving it here, may be used for callbacks later
            self.losses_train.append(fit_metrics['train']['loss_avg'])
            self.losses_valid.append(fit_metrics['valid']['total_loss'])
            self.metrics_train.append(fit_metrics['train']['stopping_loss'])
            self.metrics_valid.append(fit_metrics['valid']['stopping_loss'])

            stopping_loss = fit_metrics['valid']['stopping_loss']
            if stopping_loss < self.best_cost:
                self.best_cost = stopping_loss
                self.patience_counter = 0
                # Saving model
                self.best_network = deepcopy(self.network)
                # torch.save(self.network.state_dict(), "best_network.pth")
                has_improved = True
            else:
                self.patience_counter += 1
                has_improved=False
            self.epoch += 1
            total_time += time.time() - starting_time



            if self.verbose > 0:
                if self.epoch % self.verbose == 0:
                    separator = "|"
                    msg_epoch = f"| {self.epoch:<5} | "
                    msg_epoch += f" {fit_metrics['train']['stopping_loss']:.5f}"
                    msg_epoch += f' {separator:<2} '
                    msg_epoch += f" {fit_metrics['valid']['stopping_loss']:.5f}"
                    msg_epoch += f' {separator:<2} '
                    msg_epoch += f" {np.round(total_time, 1):<10}"
                    msg_epoch += f" {has_improved}"
                    print(msg_epoch)

        if self.verbose > 0:
            if self.patience_counter == self.patience:
                print(f"Early stopping occured at epoch {self.epoch}")
            print(f"Training done in {total_time:.3f} seconds. Best loss : {self.best_cost:.5f}")
            print('---------------------------------------')

        self.history = {"train": {"loss": self.losses_train,
                                  "metric": self.metrics_train,
                                  "lr": self.learning_rates},
                        "valid": {"loss": self.losses_valid,
                                  "metric": self.metrics_valid}}
        # load best models post training
        self.load_best_model()

        # compute feature importance once the best model is defined
        # self._compute_feature_importances(train_dataloader)

    def save_model(self, path):
        """
        Saving model with two distinct files.
        """
        saved_params = {}
        for key, val in self.get_params().items():
            if isinstance(val, type):
                # Don't save torch specific params
                continue
            else:
                saved_params[key] = val

        # Create folder
        Path(path).mkdir(parents=True, exist_ok=True)

        # Save models params
        with open(Path(path).joinpath("model_params.json"), "w", encoding="utf8") as f:
            json.dump(saved_params, f)

        # Save state_dict
        torch.save(self.network.state_dict(), Path(path).joinpath("network.pt"))
        shutil.make_archive(path, 'zip', path)
        shutil.rmtree(path)
        print(f"Successfully saved model at {path}.zip")
        return f"{path}.zip"

    def load_model(self, filepath):

        try:
            try:
                with zipfile.ZipFile(filepath) as z:
                    with z.open("model_params.json") as f:
                        loaded_params = json.load(f)
                    with z.open("network.pt") as f:
                        try:
                            saved_state_dict = torch.load(f)
                        except io.UnsupportedOperation:
                            # In Python <3.7, the returned file object is not seekable (which at least
                            # some versions of PyTorch require) - so we'll try buffering it in to a
                            # BytesIO instead:
                            saved_state_dict = torch.load(io.BytesIO(f.read()))
                            
            except:
                with open(os.path.join(filepath, "model_params.json")) as f:
                        loaded_params = json.load(f)

                saved_state_dict = torch.load(os.path.join(filepath, "network.pt"), map_location="cpu")
 
        except KeyError:
            raise KeyError("Your zip file is missing at least one component")

        #print(loaded_params)
        if torch.cuda.is_available():
            device_name = 'cuda'
        else:
            device_name = 'cpu'
        loaded_params["device_name"] = device_name
        self.__init__(**loaded_params)
        
        

        self.init_network(
            input_dim=self.input_dim,
            output_dim=self.output_dim,
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            cat_idxs=self.cat_idxs,
            cat_dims=self.cat_dims,
            cat_emb_dim=self.cat_emb_dim,
            n_independent=self.n_independent,
            n_shared=self.n_shared,
            epsilon=self.epsilon,
            virtual_batch_size=1024,
            momentum=self.momentum,
            device_name=self.device_name,
            mask_type=self.mask_type
        )
        self.network.load_state_dict(saved_state_dict)
        self.network.eval()
        return

    def fit_epoch(self, train_dataloader, valid_dataloader):
        """
        Evaluates and updates network for one epoch.
        Parameters
        ----------
            train_dataloader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
            valid_dataloader: a :class: `torch.utils.data.Dataloader`
                DataLoader with valid set
        """
        train_metrics = self.train_epoch(train_dataloader)
        valid_metrics = self.predict_epoch(valid_dataloader)

        fit_metrics = {'train': train_metrics,
                       'valid': valid_metrics}

        return fit_metrics

    @abstractmethod
    def train_epoch(self, train_loader):
        """
        Trains one epoch of the network in self.network
        Parameters
        ----------
            train_loader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
        """
        raise NotImplementedError('users must define train_epoch to use this base class')

    @abstractmethod
    def train_batch(self, data, targets):
        """
        Trains one batch of data
        Parameters
        ----------
            data: a :tensor: `torch.tensor`
                Input data
            target: a :tensor: `torch.tensor`
                Target data
        """
        raise NotImplementedError('users must define train_batch to use this base class')

    @abstractmethod
    def predict_epoch(self, loader):
        """
        Validates one epoch of the network in self.network
        Parameters
        ----------
            loader: a :class: `torch.utils.data.Dataloader`
                    DataLoader with validation set
        """
        raise NotImplementedError('users must define predict_epoch to use this base class')

    @abstractmethod
    def predict_batch(self, data, targets):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            batch_outs: dict
        """
        raise NotImplementedError('users must define predict_batch to use this base class')

    def load_best_model(self):
        if self.best_network is not None:
            self.network = self.best_network
            # self.network = self.network.load_state_dict(torch.load("best_network.pth")).to(self.device)

    @abstractmethod
    def predict(self, X):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            predictions: np.array
                Predictions of the regression problem or the last class
        """
        raise NotImplementedError('users must define predict to use this base class')

    def explain(self, X):
        """
        Return local explanation
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            M_explain: matrix
                Importance per sample, per columns.
            masks: matrix
                Sparse matrix showing attention masks used by network.
        """
        self.network.eval()

        dataloader = DataLoader(PredictDataset(X),
                                batch_size=self.batch_size, shuffle=False)

        for batch_nb, data in enumerate(dataloader):
            data = data.to(self.device).float()

            M_explain, masks = self.network.forward_masks(data)
            for key, value in masks.items():
                masks[key] = csc_matrix.dot(value.cpu().detach().numpy(),
                                            self.reducing_matrix)

            if batch_nb == 0:
                res_explain = csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                             self.reducing_matrix)
                res_masks = masks
            else:
                res_explain = np.vstack([res_explain,
                                         csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                                        self.reducing_matrix)])
                for key, value in masks.items():
                    res_masks[key] = np.vstack([res_masks[key], value])
        return res_explain, res_masks

    def _compute_feature_importances(self, loader):
        self.network.eval()
        feature_importances_ = np.zeros((self.network.post_embed_dim))
        for data, targets,_ in loader:
            data = data.to(self.device).float()
            M_explain, masks = self.network.forward_masks(data)
            feature_importances_ += M_explain.sum(dim=0).cpu().detach().numpy()

        feature_importances_ = csc_matrix.dot(feature_importances_,
                                              self.reducing_matrix)
        self.feature_importances_ = feature_importances_ / np.sum(feature_importances_)
        


class TabNetRegressor(TabModel):

    def construct_loaders(self, X_train, y_scored_train, X_valid, y_valid, weights,
                          batch_size, num_workers, drop_last):
        """
        Returns
        -------
        train_dataloader, valid_dataloader : torch.DataLoader, torch.DataLoader
            Training and validation dataloaders
        -------
        """
        if isinstance(weights, int):
            if weights == 1:
                raise ValueError("Please provide a list of weights for regression.")
        if isinstance(weights, dict):
            raise ValueError("Please provide a list of weights for regression.")



        train_dataloader, valid_dataloader = create_dataloaders(X_train,
                                                                y_scored_train,
                                                                X_valid,
                                                                y_valid,
                                                                weights,
                                                                batch_size,
                                                                num_workers,
                                                                drop_last)
        return train_dataloader, valid_dataloader

    def update_fit_params(self, X_train, y_scored_train, X_valid, y_valid, loss_fn, loss_tr,
                          weights, max_epochs, patience,batch_size, virtual_batch_size, num_workers, drop_last):


        if loss_fn is None:
            self.loss_fn = torch.nn.functional.mse_loss
        else:
            self.loss_fn = loss_fn
            self.loss_tr = loss_tr

        assert X_train.shape[1] == X_valid.shape[1], "Dimension mismatch X_train X_valid"
        self.input_dim = X_train.shape[1]

        if len(y_scored_train.shape) == 1:
            raise ValueError("""Please apply reshape(-1, 1) to your targets
                                if doing single regression.""")
        assert y_scored_train.shape[1] == y_valid.shape[1], "Dimension mismatch y_train y_valid"
        self.output_dim = y_scored_train.shape[1]

        self.updated_weights = weights

        self.max_epochs = max_epochs
        self.patience = patience
        self.batch_size = batch_size
        self.virtual_batch_size = virtual_batch_size
        # Initialize counters and histories.
        self.patience_counter = 0
        self.epoch = 0
        self.best_cost = np.inf
        self.num_workers = num_workers
        self.drop_last = drop_last

    def train_epoch(self, train_loader):
        """
        Trains one epoch of the network in self.network
        Parameters
        ----------
            train_loader: a :class: `torch.utils.data.Dataloader`
                DataLoader with train set
        """

        self.network.train()
        y_preds = []
        ys = []
        total_loss = 0

        for data, targets_scored in train_loader:
            batch_outs = self.train_batch(data, targets_scored)
            y_preds.append(batch_outs["y_preds"].cpu().detach().numpy())
            ys.append(batch_outs["y"].cpu().detach().numpy())
            total_loss += batch_outs["loss"]

        y_preds = np.vstack(y_preds)
        ys = np.vstack(ys)

        #stopping_loss = mean_squared_error(y_true=ys, y_pred=y_preds)
        # stopping_loss =log_loss_multi(ys, torch.sigmoid(torch.as_tensor(y_preds)).numpy()  )
        total_loss = total_loss / len(train_loader)

        epoch_metrics = {'loss_avg': total_loss,
                         'stopping_loss': total_loss,
                         }

        # if self.scheduler is not None:
        #     self.scheduler.step()
            
        return epoch_metrics

    def train_batch(self, data, targets_scored):
        """
        Trains one batch of data
        Parameters
        ----------
            data: a :tensor: `torch.tensor`
                Input data
            target: a :tensor: `torch.tensor`
                Target data
        """
        self.network.train()
        data = data.to(self.device).float()

        

        targets_scored = targets_scored.to(self.device).float()

        self.optimizer.zero_grad()

        output, M_loss = self.network(data)

        loss= self.loss_fn(output, targets_scored)

        loss -= self.lambda_sparse*M_loss

        loss.backward()

        if self.clip_value:
            clip_grad_norm_(self.network.parameters(), self.clip_value)

        self.optimizer.step()

        loss_value = loss.item()
        batch_outs = {'loss': loss_value,
                      'y_preds': output,
                      'y': targets_scored}
        return batch_outs

    def predict_epoch(self, loader):
        """
        Validates one epoch of the network in self.network
        Parameters
        ----------
            loader: a :class: `torch.utils.data.Dataloader`
                    DataLoader with validation set
        """
        y_preds = []
        ys = []
        self.network.eval()
        total_loss = 0

        for data, targets in loader:
            batch_outs = self.predict_batch(data, targets)
            total_loss += batch_outs["loss"]
            y_preds.append(batch_outs["y_preds"].cpu().detach().numpy())
            ys.append(batch_outs["y"].cpu().detach().numpy())

        y_preds = np.vstack(y_preds)
        ys = np.vstack(ys)

        stopping_loss = log_loss_multi(ys, torch.sigmoid(torch.as_tensor(y_preds)).numpy()  ) #mean_squared_error(y_true=ys, y_pred=y_preds)

        if self.scheduler is not None:
            self.scheduler.step(stopping_loss)

        total_loss = total_loss / len(loader)
        epoch_metrics = {'total_loss': total_loss,
                         'stopping_loss': stopping_loss}

        return epoch_metrics

    def predict_batch(self, data, targets):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            batch_outs: dict
        """
        self.network.eval()
        data = data.to(self.device).float()
        targets = targets.to(self.device).float()

        output, M_loss = self.network(data)
       
        loss = self.loss_fn(output, targets)
        loss -= self.lambda_sparse*M_loss
        loss_value = loss.item()
        batch_outs = {'loss': loss_value,
                      'y_preds': output,
                      'y': targets}
        return batch_outs

    def predict(self, X):
        """
        Make predictions on a batch (valid)
        Parameters
        ----------
            data: a :tensor: `torch.Tensor`
                Input data
            target: a :tensor: `torch.Tensor`
                Target data
        Returns
        -------
            predictions: np.array
                Predictions of the regression problem
        """
        self.network.eval()
        dataloader = DataLoader(PredictDataset(X),
                                batch_size=self.batch_size, shuffle=False)

        results = []
        for batch_nb, data in enumerate(dataloader):
            data = data.to(self.device).float()

            output, M_loss = self.network(data)
            predictions = output.cpu().detach().numpy()
            results.append(predictions)
        res = np.vstack(results)
        return res

In [36]:
device = "cuda" if torch.cuda.is_available() else "cpu"

NFOLDS = 5

save_name = "../data/tabnet-weights-public/tabnet-raw-public-step1/tabnet_raw_step1"

In [37]:
# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=seed)
tmp = scored.groupby('drug_id')[targets_scored].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_scored])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=seed)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_scored])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
train_df = train_df.merge(drug,on="sig_id")
train_df['fold'] = train_df.drug_id.map(dct1)
train_df.loc[train_df.fold.isna(),'fold'] =\
    train_df.loc[train_df.fold.isna(),'sig_id'].map(dct2)
train_df.fold = train_df.fold.astype('int8')

In [38]:

feat_cols = [c for c in train_df.columns if c not in ["sig_id","cp_type","drug_id","fold"]]

In [49]:
import math

def run_training(fold, seed):

    seed_everything(seed)
    
    
    
    train = train_df[train_df['fold'] != fold][feat_cols]
    valid = train_df[train_df['fold'] == fold][feat_cols]


    X_train, y_scored_train   = train.values, train_targets_scored.values[train.index, :]
    X_val, y_val = valid.values, train_targets_scored.values[valid.index, :]
    

    model = TabNetRegressor(
                            n_d = 28,
                            n_a = 28,
                            n_steps = 1,
                            gamma = 1.3,
                            n_independent=2,
                            n_shared=1,
                            momentum=0.02,
                            epsilon=1e-15,
                            lambda_sparse = 0,
                            optimizer_fn = optim.Adam,
                            optimizer_params = dict(lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY),
                            mask_type = "entmax",
                            scheduler_params = dict(
                                mode = "min", patience = PATIENCE_SCH, min_lr = 1e-5, factor = 0.9, verbose = True),
                            scheduler_fn = ReduceLROnPlateau,
                            seed = seed,
                            verbose = 1)

                             


    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)


    model.fit(
        X_train = X_train,
        y_scored_train=y_scored_train,
        # eval_set = [(X_val, y_val)],
        X_valid=X_val, 
        y_valid=y_val,
        # eval_name = ["val"],
        # eval_metric = ["logits_ll"],
        max_epochs = EPOCHS,
        patience = PATIENCE,
        batch_size = BATCH_SIZE, 
        virtual_batch_size = 32,
        num_workers = 1,
        drop_last = False,
        # To use binary cross entropy because this is not a regression problem
        loss_fn = F.binary_cross_entropy_with_logits
    )


    oof = np.zeros((train_df.shape[0], train_targets_scored.shape[1]))
    
    model.load_best_model()
    preds = model.predict(X_val)
    oof[valid.index] = torch.sigmoid(torch.as_tensor(preds)).detach().cpu().numpy()

    X_test = test_df[feat_cols].values
    preds = model.predict(X_test)
    predictions = torch.sigmoid(torch.as_tensor(preds)).detach().cpu().numpy()
    
    return oof, predictions

In [50]:

EPOCHS = 200 #200
PATIENCE_SCH=5
PATIENCE=20 #20
LEARNING_RATE =2e-2 #1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5

BATCH_SIZE = 1024


def run_k_fold(NFOLDS, seed):
    oof = np.zeros((train_df.shape[0], train_targets_scored.shape[1]))
    predictions = np.zeros((test_df.shape[0], train_targets_scored.shape[1]))
    
    for fold in range(NFOLDS):
        print(f"SEED {seed} - FOLD {fold}")
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions


In [51]:
# Averaging on multiple SEEDS

#SEED = [0,1,2,3,4,5,6] #<-- Update
#SEED = [0,3,6]
SEED = [0]
oof = np.zeros((train_df.shape[0], train_targets_scored.shape[1]))
predictions = np.zeros((test_df.shape[0], train_targets_scored.shape[1]))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train_df[train_targets_scored.columns] = oof
test_df[train_targets_scored.columns] = predictions

SEED 0 - FOLD 0
Device used : cuda
Will train until validation stopping metric hasn't improved in 20 rounds.
---------------------------------------
| EPOCH |  train  |   valid  | total time (s)
| 1     |  0.38753 |   0.05153 |   1.6        True
| 2     |  0.02837 |   0.02749 |   3.4        True
| 3     |  0.02377 |   0.02207 |   5.0        True
| 4     |  0.02158 |   0.02128 |   6.6        True
| 5     |  0.02112 |   0.02093 |   8.3        True
| 6     |  0.02065 |   0.02068 |   9.9        True
| 7     |  0.02030 |   0.02047 |   11.6       True
| 8     |  0.02015 |   0.02008 |   13.3       True
| 9     |  0.01955 |   0.01975 |   14.9       True
| 10    |  0.01920 |   0.01996 |   16.6       False
| 11    |  0.01911 |   0.01940 |   18.2       True
| 12    |  0.01894 |   0.02019 |   19.9       False
| 13    |  0.01875 |   0.01917 |   21.5       True
| 14    |  0.01844 |   0.01899 |   23.2       True
| 15    |  0.01834 |   0.02002 |   24.8       False
| 16    |  0.01812 |   0.02121 |   26

In [52]:
y_true = train_targets_scored.values
y_pred = train_df[train_targets_scored.columns].values

score = 0
for i in range(len(train_targets_scored.columns)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / train_targets_scored.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.017560459188415253


In [53]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_true,y_pred))

0.635720188915399


In [ ]:
#withput dropout .1
#CV log_loss:  0.01760583639383521 - 0.6293958771679892
#factor .85 CV log_loss:  0.017566634446178522 - 0.6309077494697637
#factor .9 CV log_loss:  0.017560459188415253 - 0.635720188915399

In [44]:
#CV log_loss:  0.01737219416857171 - 0.623911530267073
#without var threshold - CV log_loss:  0.01745984046908206 - 0.65454762689149
#without prelu - CV log_loss:  0.017398596719990684 - 0.6234924178905198
#28/28 - CV log_loss:  0.017367893132362852 - 0.6253214143582515
#28/28 shared 1 - CV log_loss:  0.017362615722795263 - 0.6313124750373218
#factor .85 CV log_loss:  0.01733148506163567 - 0.6320480113875424
#factor .8 CV log_loss:  0.017318972353999255 - 0.6334342588920354
    #factor .75 CV log_loss:  0.017354087985405227 - 0.6350515869886433
    #indep 3 - CV log_loss:  0.017382696307823724 - 0.6255216650265163
    #24/28 - CV log_loss:  0.01734208324672452 - 0.6313557477377315
    #28/24 - CV log_loss:  0.017345107979068086 - 0.6273993901501496
    #28/30 - CV log_loss:  0.01739716192160574 - 0.6304467367115033

In [56]:
oof_csv3 = train_features[["sig_id"]].merge(train_df[np.append(['sig_id'],train_targets_scored.columns)], on='sig_id', how='inner')

In [55]:
sub_mdl3 = sample_submission.drop(columns=train_targets_scored.columns).merge(test_df[np.append(['sig_id'],train_targets_scored.columns)], on='sig_id', how='left').fillna(0)

In [59]:
del data_all,X_test,cols_numeric,data_feats,dct1,dct2,dd,drug,feat_cols,idxT,idxV,oof,oof_,pca_cells,pca_genes,predictions,predictions_,r_gpu,sample_submission,scored,test_df,test_features,tmp,train_df,train_features,train_targets_nonscored,train_targets_scored,vc,y_pred,y_true

NameError: name 'data_all' is not defined

# 2nd model

In [77]:
# Parameters
no_ctl = True
ncompo_genes = 600
ncompo_cells = 50

In [83]:
train_features = pd.read_csv(data_dir+'/train_features.csv')
train_targets_scored = pd.read_csv(data_dir+'/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir+'/train_targets_nonscored.csv')

test_features = pd.read_csv(data_dir+'/test_features.csv')
sample_submission = pd.read_csv(data_dir+'/sample_submission.csv')
drug = pd.read_csv(data_dir+'/train_drug.csv')

In [84]:
targets_scored = train_targets_scored.columns[1:]
scored = train_targets_scored.merge(drug, on='sig_id', how='left') 

In [85]:
if no_ctl:
    # cp_type == ctl_vehicle
    print("not_ctl")
    train_features = train_features[train_features["cp_type"] != "ctl_vehicle"]
    test_features = test_features[test_features["cp_type"] != "ctl_vehicle"]
    train_targets_scored = train_targets_scored.iloc[train_features.index]
    train_targets_nonscored = train_targets_nonscored.iloc[train_features.index]
    train_features.reset_index(drop = True, inplace = True)
    test_features.reset_index(drop = True, inplace = True)
    train_targets_scored.reset_index(drop = True, inplace = True)
    train_targets_nonscored.reset_index(drop = True, inplace = True)

not_ctl


In [86]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [87]:
seed=42

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed)

In [88]:
#RankGauss
for col in (GENES + CELLS):
    transformer = QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [89]:
# GENES
n_comp = ncompo_genes 

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[train_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [90]:
#CELLS
n_comp = ncompo_cells

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [91]:
from scipy import stats

def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1) ## <==
        # df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1) ## <==
        # df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        #df['g_skew'] = df[features_g].skew(axis = 1)
        # df['g_q25'] = df[features_g].quantile(q=.25,axis = 1)
        # df['g_q50'] = df[features_g].quantile(q=.5,axis = 1)
        # df['g_q75'] = df[features_g].quantile(q=.75,axis = 1)
        #df['g_var'] = df[features_g].apply(axis=1,func=stats.variation)
        # df['g_mad'] = df[features_g].mad(axis = 1)


        df['c_sum'] = df[features_c].sum(axis = 1) ## <==
        # df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1) ## <==
        # df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        #df['c_skew'] = df[features_c].skew(axis = 1)
        # df['c_q25'] = df[features_c].quantile(q=.25,axis = 1)
        # df['c_q50'] = df[features_c].quantile(q=.5,axis = 1)
        # df['c_q75'] = df[features_c].quantile(q=.75,axis = 1)
        # df['c_var'] = df[features_c].apply(axis=1,func=stats.variation)
        # df['c_mad'] = df[features_c].mad(axis = 1)



        df['gc_sum'] = df[features_g + features_c].sum(axis = 1) ## <==
        # df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        # df['gc_std'] = df[features_g + features_c].std(axis = 1)
        # df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        # df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        # df['gc_q25'] = df[features_g + features_c].quantile(q=.25,axis = 1)
        # df['gc_q50'] = df[features_g + features_c].quantile(q=.5,axis = 1)
        # df['gc_q75'] = df[features_g + features_c].quantile(q=.75,axis = 1)
        # df['gc_var'] = df[features_g + features_c].apply(axis=1,func=stats.variation)
        # df['gc_mad'] = df[features_g + features_c].mad(axis = 1)


        
    return train, test

train_features,test_features=fe_stats(train_features,test_features)

In [92]:
from sklearn.feature_selection import VarianceThreshold


var_thresh = VarianceThreshold(0.8)  #<-- Update
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

(21948, 1047)

In [93]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
# train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target_scored = train[train_targets_scored.columns]
target_nscored = train[train_targets_nonscored.columns]

In [94]:
train = train.drop('cp_type', axis=1)
test = test_features.drop('cp_type', axis=1)

In [95]:
target_scored_cols = target_scored.drop('sig_id', axis=1).columns.values.tolist()
target_nscored_cols = target_nscored.drop('sig_id', axis=1).columns.values.tolist()

In [96]:
class TrainDataset:
    def __init__(self, features, targets_scored,targets_nscored):
        self.features = features
        self.targets_scored = targets_scored
        self.targets_nscored = targets_nscored
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y_scored' : torch.tensor(self.targets_scored[idx, :], dtype=torch.float),
            'y_nscored' : torch.tensor(self.targets_nscored[idx, :], dtype=torch.float)           
        }
        return dct

class ValidDataset:
    def __init__(self, features, targets_scored):
        self.features = features
        self.targets_scored = targets_scored
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y_scored' : torch.tensor(self.targets_scored[idx, :], dtype=torch.float),          
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

In [97]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets1, targets2 = data['x'].to(device), data['y_scored'].to(device), data['y_nscored'].to(device)
#         print(inputs.shape)
        outputs1,outputs2 = model(inputs)
        loss1 = loss_fn(outputs1, targets1)
        loss2 = loss_fn(outputs2, targets2)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y_scored'].to(device)
        outputs,_ = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        

    final_loss /= len(dataloader)


    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds_scored = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs_scored,_ = model(inputs)
        
        preds_scored.append(outputs_scored.sigmoid().detach().cpu().numpy())
        
    preds_scored = np.concatenate(preds_scored)
    
    return preds_scored

In [98]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [99]:
class Model(nn.Module):
    def __init__(self, num_features, 
                 num_targets_scored,
                 num_targets_nscored, 
                 hidden_sizes,
                 dropout_rates):

        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_sizes[0]))
        self.activation1 = torch.nn.PReLU(num_parameters = hidden_sizes[0], init = 1.0)
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_sizes[0])
        self.dropout2 = nn.Dropout(dropout_rates[0])
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_sizes[0], hidden_sizes[1]))
        self.activation2 = torch.nn.PReLU(num_parameters = hidden_sizes[1], init = 1.0)


        self.batch_norm2b = nn.BatchNorm1d(hidden_sizes[1])
        self.dropout2b = nn.Dropout(dropout_rates[1])
        self.dense2b = nn.utils.weight_norm(nn.Linear(hidden_sizes[1], hidden_sizes[2]))
        self.activation2b = torch.nn.PReLU(num_parameters = hidden_sizes[2], init = 1.0)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_sizes[2])
        self.dropout3 = nn.Dropout(dropout_rates[2])
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_sizes[2], num_targets_scored))
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_sizes[2], num_targets_nscored))

    def init_bias(self,pos_scored_rate,pos_nscored_rate):
        self.dense3.bias.data = nn.Parameter(torch.tensor(pos_scored_rate, dtype=torch.float))
        self.dense4.bias.data = nn.Parameter(torch.tensor(pos_nscored_rate, dtype=torch.float))
    
    def recalibrate_layer(self, layer):
        if(torch.isnan(layer.weight_v).sum() > 0):
            print ('recalibrate layer.weight_v')
            layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
            layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

        if(torch.isnan(layer.weight).sum() > 0):
            print ('recalibrate layer.weight')
            layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
            layer.weight += 1e-7

    def forward(self, x):
        x = self.batch_norm1(x)
        self.recalibrate_layer(self.dense1)
        x = self.activation1(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        self.recalibrate_layer(self.dense2)
        x = self.activation2(self.dense2(x))

        x = self.batch_norm2b(x)
        x = self.dropout2b(x)
        self.recalibrate_layer(self.dense2b)
        x = self.activation2b(self.dense2b(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        self.recalibrate_layer(self.dense3)
        x1 = self.dense3(x)

        self.recalibrate_layer(self.dense4)
        x2 = self.dense4(x)
        return x1,x2
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [100]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [101]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 30
#EPOCHS = 300 #200
PATIENCE=40

BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 7           
EARLY_STOPPING_STEPS = PATIENCE+5
EARLY_STOP = False

#hidden_size=1500
hidden_sizes = [1200,1000,1000]
dropout_rates = [0.2619422201258426,0.2619422201258426,0.27]
#dropout_rate = 0.2619422201258426
#dropout_rate = 0.28

In [102]:
# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=seed)
tmp = scored.groupby('drug_id')[targets_scored].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_scored])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=seed)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_scored])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
folds = train.merge(drug,on="sig_id")
folds['fold'] = folds.drug_id.map(dct1)
folds.loc[folds.fold.isna(),'fold'] =\
    folds.loc[folds.fold.isna(),'sig_id'].map(dct2)
folds.fold = folds.fold.astype('int8')

In [103]:
feature_cols = [c for c in process_data(train).columns if c not in (target_scored_cols + target_nscored_cols)]
feature_cols = [c for c in feature_cols if c not in ['fold','sig_id']]
len(feature_cols)

1048

In [104]:
num_features=len(feature_cols)
num_targets_scored=len(target_scored_cols)
num_targets_nscored=len(target_nscored_cols)

In [105]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target_scored.shape)
print(target_nscored.shape)
print(sample_submission.shape)

(21948, 1654)
(21948, 1656)
(3624, 1046)
(21948, 207)
(21948, 403)
(3982, 207)


In [106]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)

    
    trn_idx = train[train['fold'] != fold].index
    val_idx = train[train['fold'] == fold].index
    
    train_df = train[train['fold'] != fold].reset_index(drop=True)
    valid_df = train[train['fold'] == fold].reset_index(drop=True)
    
    x_train, y_scored_train, y_nscored_train  = train_df[feature_cols].values, train_df[target_scored_cols].values, train_df[target_nscored_cols].values
    x_valid, y_scored_valid =  valid_df[feature_cols].values, valid_df[target_scored_cols].values
    
    train_dataset = TrainDataset(x_train, y_scored_train, y_nscored_train)
    valid_dataset = ValidDataset(x_valid, y_scored_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets_scored=num_targets_scored,
        num_targets_nscored=num_targets_nscored,
        hidden_sizes=hidden_sizes,
        dropout_rates=dropout_rates,
    )

    # model.init_bias(pos_scored_rate,pos_nscored_rate)

    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,pct_start=0.1, div_factor=1e4, final_div_factor=1e5,
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))


    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,mode = "min", patience = PATIENCE, min_lr = 1e-6, factor = 0.9)
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
   
    oof = np.zeros((len(train), target_scored.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}")
        #scheduler.step(valid_loss)
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_m1_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets_scored=num_targets_scored,
        num_targets_nscored=num_targets_nscored,
        hidden_sizes=hidden_sizes,
        dropout_rates=dropout_rates
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_m1_.pth"))
    model.to(DEVICE)
    
    predictions_scored = np.zeros((len(test_), target_scored.iloc[:, 1:].shape[1]))
    predictions_scored = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions_scored

In [107]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_scored_cols)))
    predictions_scored = np.zeros((len(test), len(target_scored_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_scored_, pred_nscored_ = run_training(fold, seed)
        
        predictions_scored += pred_scored_ / NFOLDS
        oof += oof_
        
    return oof, predictions_scored

In [108]:
# Averaging on multiple SEEDS

#SEED = [0,1,2,3,4,5,6] #<-- Update
SEED = [0,3,6]
#SEED = [0]
oof = np.zeros((len(train), len(target_scored_cols)))
predictions_scored = np.zeros((len(test), len(target_scored_cols)))



# mean_scored = np.mean(train[target_scored_cols].values,axis=0)
# mean_nscored = np.mean(train[target_nscored_cols].values,axis=0)
# pos_scored_rate = np.log(np.where(mean_scored==0, 1e-8, mean_scored))
# pos_nscored_rate = np.log(np.where(mean_nscored==0, 1e-8, mean_nscored))

for seed in SEED:
    
    oof_, predictions_scored_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions_scored += predictions_scored_ / len(SEED)

train[target_scored_cols] = oof
test[target_scored_cols] = predictions_scored

FOLD: 0, EPOCH: 0, train_loss: 1.0963618820281447, valid_loss: 0.042640757113695145
FOLD: 0, EPOCH: 1, train_loss: 0.03996050526463502, valid_loss: 0.02177127845585346
FOLD: 0, EPOCH: 2, train_loss: 0.03299188109215449, valid_loss: 0.022824962437152863
FOLD: 0, EPOCH: 3, train_loss: 0.03099722499531266, valid_loss: 0.01858476661145687
FOLD: 0, EPOCH: 4, train_loss: 0.03017753182683845, valid_loss: 0.023675367161631586
FOLD: 0, EPOCH: 5, train_loss: 0.02983695755687517, valid_loss: 0.01920233152806759
FOLD: 0, EPOCH: 6, train_loss: 0.029450410800809797, valid_loss: 0.018869943097233773
FOLD: 0, EPOCH: 7, train_loss: 0.029314227590991837, valid_loss: 0.018598442375659944
FOLD: 0, EPOCH: 8, train_loss: 0.029221118578838336, valid_loss: 0.01855507604777813
FOLD: 0, EPOCH: 9, train_loss: 0.029058650148579397, valid_loss: 0.018243699595332147
FOLD: 0, EPOCH: 10, train_loss: 0.029042176940050478, valid_loss: 0.018125043362379075
FOLD: 0, EPOCH: 11, train_loss: 0.0289603291910041, valid_loss: 

In [109]:
valid_results = train_targets_scored.drop(columns=target_scored_cols).merge(train[['sig_id']+target_scored_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_scored_cols].values
y_pred = valid_results[target_scored_cols].values

score = 0
for i in range(len(target_scored_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target_scored.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.017139895148700925


In [110]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_true,y_pred))

0.6272815022034747


In [111]:
oof_csv1 = train_features[["sig_id"]].merge(train[train_targets_scored.columns], on='sig_id', how='inner')

In [112]:
sub_mdl1 = sample_submission[["sig_id"]].merge(test[train_targets_scored.columns], on='sig_id', how='left').fillna(0)

In [131]:
ratios= [.6,.4]

train = oof_csv1.copy()
train[target_scored_cols] =  ratios[0]*oof_csv1[target_scored_cols] \
                            + ratios[1]*oof_csv3[target_scored_cols]

In [132]:
valid_results = train_targets_scored.drop(columns=target_scored_cols).merge(train[['sig_id']+target_scored_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_scored_cols].values
y_pred = valid_results[target_scored_cols].values

score = 0
for i in range(len(target_scored_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target_scored.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.01692581279316073


In [135]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_true,y_pred))

0.6257775449528624


In [134]:
sub = sub_mdl1.copy()
test[target_scored_cols]  =  ratios[0]*sub_mdl1[target_scored_cols] \
                            + ratios[1]*sub_mdl3[target_scored_cols]


sub = sample_submission[["sig_id"]].merge(test[train_targets_scored.columns], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)